In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

2022-07-19 16:23:58.947864: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-19 16:23:58.947885: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-07-19 16:23:58.947889: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-07-19 16:23:58.947933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-07-19 16:23:58.947952: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-07-19 16:23:58.948428: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F 

In [5]:
model_tf = TFT5ForConditionalGeneration.from_pretrained('mesolitica/t5-super-tiny-finetuned-noisy-ms-en')

Downloading:   0%|          | 0.00/48.5M [00:00<?, ?B/s]

2022-07-19 16:24:06.919275: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at mesolitica/t5-super-tiny-finetuned-noisy-ms-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
tokenizer = T5Tokenizer.from_pretrained('mesolitica/t5-super-tiny-finetuned-noisy-ms-en')

In [7]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
left = [unidecode(s) for s in ms]
right = [unidecode(s) for s in eng]

In [8]:
len(ms), len(eng)

(998, 998)

In [9]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Melayu ke Inggeris: {s}', return_tensors = 'tf')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_tf.generate(**padded, max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [01:10<00:00,  1.13s/it]


In [10]:
len(results)

998

In [11]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [12]:
refs = [filtered_right]
sys = filtered_left

In [13]:
r = bleu.corpus_score(sys, refs)

In [14]:
r.__dict__

{'name': 'BLEU',
 'score': 35.336854036912634,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '67.8/42.9/29.3/20.4 (BP = 0.973 ratio = 0.974 hyp_len = 22953 ref_len = 23570)',
 'bp': 0.9734770611426504,
 'counts': [15573, 9413, 6145, 4064],
 'totals': [22953, 21956, 20959, 19962],
 'sys_len': 22953,
 'ref_len': 23570,
 'precisions': [67.84734021696511,
  42.87210785206777,
  29.31914690586383,
  20.358681494840198],
 'prec_str': '67.8/42.9/29.3/20.4',
 'ratio': 0.9738226559185406}

In [15]:
chrf.corpus_score(sys, refs)

chrF2++ = 59.12